<a href="https://colab.research.google.com/github/leekate/titanic_pro/blob/master/Titanic04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kaggle: Titanic_Machine Learning from Disaster
#### 목표:트레이닝 세트에서 패턴을 찾고 살아남은 사람을 예측하기

In [ ]:
#데이터 확인에 필요한 라이브러리 불러오기
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

# 1. 데이터 확인부터 하자

In [ ]:
train_data = pd.read_csv("train.csv")
train_data.head()

In [ ]:
test_data = pd.read_csv("test.csv")
test_data.head()

항목별 의미이다.
*   Survived: 0 = 아니오, 1 = 예
*   pclass: 객실 클래스 / 1 = 1등석, 2 = 2등석, 3 = 3등석 
*   Name : 탑승자 이름
*   Sex : 탑승자 성별
*   Age : 탑승자 나이
*   pclass: 티켓 클래스 / 1 = 1위, 2 = 2위, 3 = 3위
*   SibSp: 타이타닉호에 탑승한 형제/배우자 수
*   parch: 타이타닉호에 승선한 부모/아이들의 수
*   TIcket: 티켓 번호
*   Fare : 여객운임
*   Cabin: 객실 번호
*   Embarked: 탑승 항만 / C = 체르부르, Q = 퀸스타운, S = 사우샘프턴

## 간단하게 bar로 표현
몇등석에 있었는지에 따라 살아남은 정도가 궁금해졌다. 좋은 1등석에 있을수록 많은 사람이 살아남았을까?

In [ ]:
#교차표의 행 합, 열 합을 추가해서 한눈에 알아보자
#pd.crosstab(margins=True)

pd.crosstab(train_data['Pclass'],train_data['Survived'], margins=True)

survived 1이 생존한 샘플이다. 생존한 사람의 수만 보았을 때 1등석이 135명, 3등석이 119명, 2등석이 87명으로 많다. 하지만 생존자의 수만 보는 것보다 비율을 생각하는 것이 더 중요하다. 

1등석 이용객 216명 중 136명이 살아남았으니 생존비율은 136/216=0.63, 2등석 이용객 184명 중 87명이 살아남았으니 생존비율은 87/184=0.47, 3등석 이용객 491명 중 119명이 살아남았으므로 생존비율은 119/491=0.24이다. 따라서 1등석, 2등석, 3등석 순으로 생존비율이 높음을 알수 있다.


하이 레벨의 공간을 이용한 사람일수록 생존율이 높다. 왜 그럴까? 구조가 탈출하기 더 쉬웠거나 탈출 상황에서의 대처가 달랐나? 

margin 대신 normalize를 이용해 전체 중 각 항목의 비율도 알 수 있다.

In [ ]:
pd.crosstab(train_data['Pclass'],train_data['Survived'], normalize=True)

3등급을 이용한 사람 중 사망한 사람은 0.42%로 굉장히 많은 비율을 차지 하고 있다.

이처럼 알고싶었던 정보를 쉽게 알 수 있다. 성별과도 비교해보자.

In [ ]:
pd.crosstab(train_data['Sex'],train_data['Survived'], margins=True)


여성의 생존률이 남성의 생존률보다 크다. (233/314와 109/577) 왜 그럴까?
사회적 약자라고 인식되는 여성이 탈출에 있어 우선사항이 되었기 때문이라고 생각한다. 

그렇다면 나이로 구분되는 사회적 약자도 생존률이 높을까?

In [ ]:
#나이 데이터는 24,27,... 너무 다양해서 표현하기 어렵다. 
#따라서 아래 그래프처럼 표현하는 게 눈에 더 잘 들어온다.
import seaborn as sns

plt.style.use('seaborn')
sns.set(font_scale=2.5)

fig, ax = plt.subplots(1, 1, figsize=(9, 5))
sns.kdeplot(train_data[train_data['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(train_data[train_data['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived', 'Dead'])
plt.show()

사회적 약자... 나이 파트에서 잘 이루어졌는지는 모르겠다

앗 그런데 pandas를 확인했을 때 비어보이는 값들이 많다. missing data를 파악해보자

In [ ]:
test_data.info()

여기까지 확인했을 때 cabin의 값을 가지는 샘플이 많지 않다는 것을 알 수 있다. 샘플이 부족하면 언더피팅이 발생할 수 있어 샘플을 채워주거나 모델을 복잡하게 하거나 규제를 줄여 언더피팅 발생을 막아야 한다. 하지만 cabin이라는 항목이 값을 채워주기에는 너무 많은 양이 존재하지 않고, 살아남은 사람을 예상하는 것과 큰 연관이 없다고 생각해 이 데이터를 빼고 생각하려고 한다.

In [ ]:

train_new=train_data.drop('Cabin',axis=1)

train_new

성별이 문자형이므로 다루기 쉽게 숫자형으로 바꾸어주자

In [ ]:

from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()

In [ ]:
train_new = encoder.fit_transform(train_new['Sex'])

In [ ]:
train_new['Sex']

In [ ]:
train_new.Sex.value_counts().plot.bar()

단일 특성을 가지고 생존 확률을 예측하기보단 여러가지 특성을 종합해서 예측을 하는 것이 더 좋을 것이다.

# 2. 데이터 전처리를 해보자

데이터를 파악해보고 이상한 값들을 수정하고, 결측치를 채워주고, 계산을 위해 알맞은 형태로 바꾸어주자.

## Embarked 파트

In [ ]:
import numpy as np
train_new.dropna(subset=['Embarked'],inplace=True)
train_new.isnull().sum()

In [ ]:
import numpy as np
test_data.dropna(subset=['Embarked'],inplace=True)
test_data.isnull().sum()

## Sex 파트

앞에서 스트링으로 되어 있는 데이터의 형태를 숫자로 바꾸어주는 과정을 거쳤기 때문에 여기서는 test세트만 처리하자.

In [ ]:
test_data.Sex = test_data.Sex.replace('female',0)
test_data.Sex = test_data.Sex.replace('male',1)

### Age 파트

age 전처리도 해보자. age는 중간값으로 결측치를 채워주자!

In [ ]:
train_new['Age'].fillna(train_new['Age'].median(), inplace = True)


test데이터도 처리해준다.

In [ ]:
test_data['Age'].fillna(test_data['Age'].median(), inplace = True)

### Fare 파트

fare 전처리도 할거다. 이 친구도 중간값을 이용해 결측치를 채워주자!

In [ ]:
train_new['Fare'].fillna(train_new['Fare'].median(), inplace = True)



In [ ]:
test_data['Fare'].fillna(test_data['Fare'].median(), inplace = True)

### 나머지 파트


이름, 승객번호, 캐빈, 티켓번호는 생존과 관련이 없다고 생각한다. 그러므로 앞에서 캐빈을 삭제한 거처럼 마저 삭제해버리자.

In [ ]:
drop_column = ['PassengerId', 'Ticket',"Name"]
train_new.drop(drop_column, axis=1, inplace = True)



test데이터에서도 똑같이 처리해주자!


In [ ]:

drop_column = ['Name','PassengerId', 'Ticket']
test_data.drop(drop_column, axis=1, inplace = True)

In [ ]:

drop_column = ['Cabin']
test_data.drop(drop_column, axis=1, inplace = True)

## 특성 스케일링

데이터에 적용할 가장 중요한 변환 중 하나는 **특성 스케일링**이다. 모든 특성의 범위를 같게 만들어주는 것은 
1. min-max 스케일링 (Normalization)
2. 표준화
이다. 



In [ ]:
np.mean(train_new['Fare']), np.std(train_new['Fare'])

In [ ]:
train_new_fare = np.asanyarray(train_new['Fare'])
X = (train_new_fare.reshape(-1, 1) )
pd.DataFrame(X).describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X)
X_scaled = scaler.transform(X)
np.mean(X_scaled), np.std(X_scaled)

In [ ]:
train_new['Fare']=X_scaled
np.mean(train_new['Fare']), np.std(train_new['Fare'])

test데이터에도 적용

In [ ]:
np.mean(test_data['Fare']), np.std(test_data['Fare'])

In [ ]:
test_data_fare = np.asanyarray(test_data['Fare'])
Y = (test_data_fare.reshape(-1, 1) )
pd.DataFrame(Y).describe()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(Y)
Y_scaled = scaler.transform(Y)
np.mean(Y_scaled), np.std(Y_scaled)

In [ ]:
test_data['Fare']=Y_scaled
np.mean(test_data['Fare']), np.std(test_data['Fare'])

In [ ]:
test_data['Fare']

엥 조금 이상하지만 진행해보자

## 정제된 데이터 확인

In [ ]:
print('Train columns with null values: \n', train_new.isnull().sum())
print("="*10)
print (train_new.info())
print("="*10)

train_new




test 세트에서의 정제를 확인해보자

In [ ]:
print('Train columns with null values: \n', test_data.isnull().sum())
print("="*10)
print (test_data.info())
print("="*10)

test_data



아주 좋다. 모델링을 하기 위한 준비 운동이 끝났다. 여기까지 했다면 거의 다 한 것이나 다름없다. 이제 모델링을 시작해보자!

# 모델링 및 학습

어떤 알고리즘을 선택해야 할까?

decision tree, 랜덤포레스트, 배깅, 부스팅을해보겠다. 내가 적용할 수 있는 알고리즘 중 가장 최선의 방법이다.




## Decision Tree

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from IPython.display import Image
import pandas as pd
import numpy as np
import pydotplus
import os

In [ ]:
A = np.array(pd.DataFrame(train_new, columns=['Sex','Name', 'Embarked', 'Fare']))
B = np.array(pd.DataFrame(train_new, columns=['Survived']))

In [ ]:
A_train_new, A_test_new, B_train_new, B_test_new = train_test_split(A,B,test_size=0.1)

In [ ]:
my_clf=DecisionTreeClassifier()

In [ ]:
my_clf=my_clf.fit(A_train_new,B_train_new)

앗 전처리가 완벽하지 않았다. Embarked 수정해줘야한다.

In [ ]:
train_new.Embarked = train_new.Embarked.replace('S',3)
train_new.Embarked = train_new.Embarked.replace('Q',4)
train_new.Embarked = train_new.Embarked.replace('C',5)

In [ ]:
A = np.array(pd.DataFrame(train_new, columns=['Sex','Name', 'Embarked', 'Fare']))
B = np.array(pd.DataFrame(train_new, columns=['Survived']))

A_train_new, A_test_new, B_train_new, B_test_new = train_test_split(A,B,test_size=0.1)

my_clf=DecisionTreeClassifier()

my_clf=my_clf.fit(A_train_new,B_train_new)

## RandomForest

## Bagging

## Boosting

## SVC